In [ ]:
import itertools as it
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.decomposition import KernelPCA
from sklearn.manifold import TSNE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split

from possibilearn import flatten

rs = 20190105

In [ ]:
import sys
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Create STDERR handler
handler_video = logging.StreamHandler(sys.stderr)
# Create file handler
handler_file = logging.FileHandler('axiom-classification.log')

# Create formatter and add it to the handler
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler_video.setFormatter(formatter)
handler_file.setFormatter(formatter)

# Set STDERR handler as the only handler 
logger.handlers = [handler_video, handler_file]

In [ ]:
mu = np.load('mu.npy')

In [ ]:
gram_jaccard = np.load('jaccard_similarity.npy')
dist_jaccard = 1 - gram_jaccard

In [ ]:
dist_length = np.load('length_distance.npy')
gram_length = 1 - dist_length

In [ ]:
dist_leven = np.load('levenshtein_distance.npy')
gram_leven = 1 - dist_leven

In [ ]:
dist_hamming = np.load('hamming_distance.npy')
gram_hamming = 1 - dist_hamming

In [ ]:
def alpha_cut(alpha):
    return lambda x: 1 if x >= alpha else 0


def cut_labels(mu, alpha):
    return np.array(list(map(alpha_cut(alpha), mu)))

In [ ]:
reduced_data_dir = 'data/reduction/'

def learn_experiment(data_matrix,
                     reduction_procedure, reduced_data_filename,
                     mu,
                     learning_algorithm_factory,
                     d=2,
                     alpha_levels=(.2, .4, .5, .75, .9),
                     num_holdouts=5,
                     percentages=(.8, 0, .2),
                     hyperparams_set = {},
                     validation_metric = metrics.accuracy_score,
                     test_metric = metrics.accuracy_score,
                     verbose=False):
    
    # this is to check that any time we have no parameters the validation
    # percentage is 0 and vice versa
    
    assert((hyperparams_set!={} or percentages[1]==0) and \
           (percentages[1]!=0 or hyperparams_set=={}))
    
    hyperparams_iterator = ParameterGrid(hyperparams_set)
    
    
    reduced_data_fullname = reduced_data_dir + reduced_data_filename + '.npy'
    if os.path.isfile(reduced_data_fullname):
        logger.info('Found cached reduced data. Retrieving {}'.format(reduced_data_filename))
        X = np.load(reduced_data_fullname)
    else:
        logger.info('Performing reduction procedure...')
        X = reduction_procedure(n_components=d,
                                random_state=rs).fit_transform(data_matrix)
    
        logger.info('Done!')
        np.save(reduced_data_fullname, X)
        
    label_set = [cut_labels(mu, alpha) for alpha in alpha_levels]
    
    n = len(X)
    assert(n==len(mu))

    ##paired_X = [X[i:i+2] for i in range(0, n, 2)]
    
    best_test_median = float('inf')
    best_test_metric = []
    
    for alpha, y in zip(alpha_levels, label_set):
        
        logger.info('Checking alpha={:.2f}'.format(alpha))
    
        ##paired_y = [y[i:i+2] for i in range(0, n, 2)]

        metric_train = []
        metric_test = []

        for h in range(num_holdouts):
            (X_train,
             X_validate_test,
             y_train,
             y_validate_test) = train_test_split(X, y,
                                                 train_size=percentages[0],
                                                 test_size=1-percentages[0],
                                                 stratify=y)
            
            if percentages[1]==0: # no validation set
                X_validate = []
                X_test = X_validate_test
                y_validate = []
                y_test = y_validate_test
            else:
                val_perc_rel = percentages[1]/(percentages[1]+percentages[2])
                (X_validate,
                 X_test,
                 y_validate,
                 y_test) = train_test_split(X_validate_test, y_validate_test,
                                            train_size=val_perc_rel,
                                            test_size=1-val_perc_rel,
                                            stratify=y_validate_test)

            logger.info('holdout {} of {}'.format(h+1, num_holdouts))
            
            best_hyperparams = {}
            best_err = np.inf
            
            for hyperparams in hyperparams_iterator:
                if not hyperparams:
                    break
                    
                #logger.info('Checking {}'.format(hyperparams))
                
                learning_algorithm = learning_algorithm_factory(**hyperparams)
                learning_algorithm.fit(X_train, y_train)
                y_pred = learning_algorithm.predict(X_validate)
                error = validation_metric(y_validate, y_pred)
                if error < best_err:
                    best_err = error
                    best_hyperparams = hyperparams
            
            logger.info('Learning with best '
                        'hyperparams: {}'.format(best_hyperparams))
            learning_algorithm = learning_algorithm_factory(**best_hyperparams)
            if percentages[1] == 0: # no validation set
                X_train_val = X_train
                y_train_val = y_train
            else:
                X_train_val = np.vstack([X_train, X_validate])
                y_train_val = np.hstack([y_train, y_validate])
            assert(len(X_train_val) == len(X_train) + len(X_validate))
            assert(len(y_train_val) == len(y_train) + len(y_validate))
            learning_algorithm.fit(X_train_val, y_train_val)
            pred_train = learning_algorithm.predict(X_train_val)
            pred_test = learning_algorithm.predict(X_test)
            
            metric_train.append(test_metric(y_train_val, pred_train))
            metric_test.append(test_metric(y_test, pred_test))
        
        test_median = np.median(metric_test)
        if test_median < best_test_median:
            best_test_median = test_median
            best_test_metric = [alpha,
                                np.mean(metric_train),
                                np.median(metric_train),
                                np.std(metric_train),
                                np.mean(metric_test),
                                test_median,
                                np.std(metric_test)]
    names = ['alpha',
             'train_mean', 'train_median', 'train_std',
             'test_mean','test_median', 'test_std']
    return dict(zip(names, best_test_metric))

In [ ]:
def SVC_custom(*args, **kwargs):
    return SVC(*args, **kwargs, max_iter=5000)

from sklearn.preprocessing import MinMaxScaler
        
class ScaledSVC:
    def __init__(self, *args, **kwargs):
        self.svc = SVC(*args, **kwargs, max_iter=5000)
        self.scaler = MinMaxScaler()
    
    def fit(self, X, y):
        self.scaler.fit(X)
        return self.svc.fit(self.scaler.transform(X), y)
    
    def predict(self, X):
        self.scaler.fit(X)
        return self.svc.predict(self.scaler.transform(X))

def RandomForestClassifier_custom(*args, **kwargs):
    return RandomForestClassifier(*args, **kwargs, n_estimators=100)

def MLPClassifier_custom(*args, **kwargs):
    return MLPClassifier(*args, **kwargs, max_iter=5000)

In [ ]:
import json
import os

result_dir = './data/classification/'

In [ ]:
# considered learning algorithms
names = ['Decision tree', 'Random forest', 'Naive Bayes', 'LDA', 'MLP',
         'SVC (linear)', 'SVC (gaussian)']
algorithms = [DecisionTreeClassifier, RandomForestClassifier,
              GaussianNB, LinearDiscriminantAnalysis, MLPClassifier_custom,
              ScaledSVC, ScaledSVC]
hyperp_sets = [{'criterion': ['gini', 'entropy'], 'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                'max_features': [None, 'sqrt', 'log2'], 'max_depth': [None, 2, 5, 10]},
               {'n_estimators': [5, 10, 50, 100, 200], 'criterion': ['gini', 'entropy'], 'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                'max_features': [None, 'sqrt', 'log2'], 'max_depth': [None, 2, 5, 10]},
               {}, {}, {'hidden_layer_sizes': [[2], [4], [6], [10], [20]]},
               {'C': [.001, .01, .1, 1, 10], 'kernel': ['linear']},
               {'C': [.001, .01, .1, 1, 10], 'kernel': ['rbf'], 'gamma': [.001, .01, .1, 1, 10, 30]}]
percentages = [(.8, .1, .1), (.8, .1, .2), (.8, 0, .2), (.8, 0, .2),
               (.8, .1, .1),(.8, .1, .1), (.8, .1, .1)]

# considered data matrices
data_names = ['hamming', 'jaccard', 'length', 'levenshtein']
data_matrices = [gram_hamming, gram_jaccard, gram_length, gram_leven]

# considered number of extracted components
components = [2, 3, 5, 10, 30]

In [15]:
num_experiments = len(algorithms) *  len(data_matrices) *  len(components)

learning_procedures = zip(names, algorithms, hyperp_sets, percentages)

data = zip(data_names, data_matrices)

experiments = it.product(learning_procedures, data, components)

performed_experiments = 0

re_generate_files = False
append = True

num_groups = 10
for g in range(num_groups):
    logger.info('Started experiment group {} of {}'.format(g+1, num_groups))
    for experiment in experiments:
        learning_procedure = experiment[0]
        data_item = experiment[1]
        comp = experiment[2]
        out_file = result_dir + 'class-' + \
                   learning_procedure[0].replace(' ', '_') + '-' + \
                   'pca-' + \
                   data_item[0] + '-' + \
                   str(comp) + '.json'
        logger.info('Considering {}'.format(out_file))
        if os.path.isfile(out_file) and not re_generate_files and not append:
            logger.info('already exists, skipping')
        else:
            access_flag = 'w' if not append else 'a'
            if append:
                logger.info('appending to existing file')
            elif re_generate_files:
                logger.info('re-executing experiment')
            else:
                logger.info('does not exist, starting experiment')
            # perform experiment
            res = learn_experiment(data_item[1],
                                   KernelPCA, 'PCA-' + experiment[1][0],
                                   mu,
                                   learning_procedure[1],
                                   d=comp,
                                   alpha_levels=(.2, .4, .5, .6, .8, .9),
                                   percentages=learning_procedure[3],
                                   hyperparams_set = learning_procedure[2])
            
            if access_flag == 'a':
                logger.info('opening result file in append')
            else:
                logger.info('creating result file')
            with open(out_file, access_flag) as f:
                f.write('\n')
                json.dump(res, f)

            logger.info('experiment finished')
        performed_experiments += 1
        perc_complete = 100 * performed_experiments / num_experiments
        logger.info('completed {:.2f}%'.format(perc_complete))
    logger.info('Ended experiment group {} of {}'.format(g+1, num_groups))

2020-01-24 16:49:27,286 - INFO - Started experiment group 1 of 10
2020-01-24 16:49:27,287 - INFO - Considering ./data/classification/class-Decision_tree-pca-hamming-2.json
2020-01-24 16:49:27,288 - INFO - appending to existing file
2020-01-24 16:49:27,288 - INFO - Found cached reduced data. Retrieving PCA-hamming
2020-01-24 16:49:27,297 - INFO - Checking alpha=0.20
2020-01-24 16:49:27,303 - INFO - holdout 1 of 5
2020-01-24 16:49:27,483 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': 2}
2020-01-24 16:49:27,487 - INFO - holdout 2 of 5
2020-01-24 16:49:27,653 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': 2}
2020-01-24 16:49:27,657 - INFO - holdout 3 of 5
2020-01-24 16:49:27,847 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:49:27,851 - INFO - ho

2020-01-24 16:49:34,158 - INFO - holdout 2 of 5
2020-01-24 16:49:34,337 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': 2, 'max_features': None, 'max_leaf_nodes': None}
2020-01-24 16:49:34,342 - INFO - holdout 3 of 5
2020-01-24 16:49:34,521 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': 2, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:49:34,526 - INFO - holdout 4 of 5
2020-01-24 16:49:34,732 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:49:34,736 - INFO - holdout 5 of 5
2020-01-24 16:49:34,913 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 5}
2020-01-24 16:49:34,916 - INFO - Checking alpha=0.50
2020-01-24 16:49:34,919 - INFO - holdout 1 of 5
2020-01-24 16:49:35,105 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'm

2020-01-24 16:49:41,802 - INFO - holdout 5 of 5
2020-01-24 16:49:41,982 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': 2, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:49:41,984 - INFO - Checking alpha=0.60
2020-01-24 16:49:41,988 - INFO - holdout 1 of 5
2020-01-24 16:49:42,181 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': 2, 'max_features': 'sqrt', 'max_leaf_nodes': 10}
2020-01-24 16:49:42,187 - INFO - holdout 2 of 5
2020-01-24 16:49:42,360 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': 2, 'max_features': 'log2', 'max_leaf_nodes': 10}
2020-01-24 16:49:42,364 - INFO - holdout 3 of 5
2020-01-24 16:49:42,550 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': 2}
2020-01-24 16:49:42,554 - INFO - holdout 4 of 5
2020-01-24 16:49:42,734 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': 2

2020-01-24 16:49:48,593 - INFO - holdout 2 of 5
2020-01-24 16:49:48,772 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:49:48,776 - INFO - holdout 3 of 5
2020-01-24 16:49:48,959 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': 2, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:49:48,963 - INFO - holdout 4 of 5
2020-01-24 16:49:49,146 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': 2}
2020-01-24 16:49:49,150 - INFO - holdout 5 of 5
2020-01-24 16:49:49,335 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': 2}
2020-01-24 16:49:49,337 - INFO - Checking alpha=0.90
2020-01-24 16:49:49,340 - INFO - holdout 1 of 5
2020-01-24 16:49:49,534 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': Non

2020-01-24 16:49:56,363 - INFO - holdout 5 of 5
2020-01-24 16:49:56,589 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:49:56,591 - INFO - opening result file in append
2020-01-24 16:49:56,592 - INFO - experiment finished
2020-01-24 16:49:56,592 - INFO - completed 3.57%
2020-01-24 16:49:56,593 - INFO - Considering ./data/classification/class-Decision_tree-pca-jaccard-2.json
2020-01-24 16:49:56,593 - INFO - appending to existing file
2020-01-24 16:49:56,594 - INFO - Found cached reduced data. Retrieving PCA-jaccard
2020-01-24 16:49:56,598 - INFO - Checking alpha=0.20
2020-01-24 16:49:56,602 - INFO - holdout 1 of 5
2020-01-24 16:49:56,722 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': 2, 'max_features': 'sqrt', 'max_leaf_nodes': None}
2020-01-24 16:49:56,726 - INFO - holdout 2 of 5
2020-01-24 16:49:56,847 - INFO - Learning with best hyperparams: {'criterion': 'gini

...

2020-01-24 16:50:47,045 - INFO - holdout 1 of 5
2020-01-24 16:50:47,297 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:50:47,302 - INFO - holdout 2 of 5
2020-01-24 16:50:47,572 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:50:47,577 - INFO - holdout 3 of 5
2020-01-24 16:50:47,833 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': None}
2020-01-24 16:50:47,841 - INFO - holdout 4 of 5
2020-01-24 16:50:48,073 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:50:48,078 - INFO - holdout 5 of 5
2020-01-24 16:50:48,315 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': 5}
20

2020-01-24 16:50:56,677 - INFO - holdout 3 of 5
2020-01-24 16:50:56,960 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None}
2020-01-24 16:50:56,969 - INFO - holdout 4 of 5
2020-01-24 16:50:57,242 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': 2, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:50:57,251 - INFO - holdout 5 of 5
2020-01-24 16:50:57,535 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:50:57,538 - INFO - Checking alpha=0.50
2020-01-24 16:50:57,543 - INFO - holdout 1 of 5
2020-01-24 16:50:57,823 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': 5}
2020-01-24 16:50:57,830 - INFO - holdout 2 of 5
2020-01-24 16:50:58,101 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': 2,

2020-01-24 16:51:07,235 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': 2}
2020-01-24 16:51:07,238 - INFO - Checking alpha=0.60
2020-01-24 16:51:07,245 - INFO - holdout 1 of 5
2020-01-24 16:51:07,529 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': 2, 'max_features': 'sqrt', 'max_leaf_nodes': 100}
2020-01-24 16:51:07,538 - INFO - holdout 2 of 5
2020-01-24 16:51:07,806 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': 5}
2020-01-24 16:51:07,813 - INFO - holdout 3 of 5
2020-01-24 16:51:08,058 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': None}
2020-01-24 16:51:08,067 - INFO - holdout 4 of 5
2020-01-24 16:51:08,335 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes'

2020-01-24 16:51:17,394 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None}
2020-01-24 16:51:17,399 - INFO - holdout 3 of 5
2020-01-24 16:51:17,626 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:51:17,631 - INFO - holdout 4 of 5
2020-01-24 16:51:17,866 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None}
2020-01-24 16:51:17,877 - INFO - holdout 5 of 5
2020-01-24 16:51:18,109 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': 2}
2020-01-24 16:51:18,112 - INFO - Checking alpha=0.90
2020-01-24 16:51:18,120 - INFO - holdout 1 of 5
2020-01-24 16:51:18,359 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_node

KeyboardInterrupt: 

In [27]:
def TSNE_custom(*args, **kwargs):
    return TSNE(*args, **kwargs, method='exact')



algorithms = [DecisionTreeClassifier, RandomForestClassifier,
              GaussianNB, LinearDiscriminantAnalysis, MLPClassifier_custom,
              ScaledSVC, ScaledSVC]

hyperp_sets = [{'criterion': ['gini', 'entropy'], 'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                'max_features': [None, 'sqrt', 'log2'], 'max_depth': [None, 2, 5, 10]},
               {'n_estimators': [5, 10, 50, 100, 200], 'criterion': ['gini', 'entropy'], 'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                'max_features': [None, 'sqrt', 'log2'], 'max_depth': [None, 2, 5, 10]},
               {}, {}, {'hidden_layer_sizes': [[2], [4], [6], [10], [20]]},
               {'C': [.001, .01, .1, 1, 10], 'kernel': ['linear']},
               {'C': [.001, .01, .1, 1, 10], 'kernel': ['rbf'], 'gamma': [.001, .01, .1, 1, 10, 30]}]
percentages = [(.8, .1, .1), (.8, .1, .2), (.8, 0, .2), (.8, 0, .2),
               (.8, .1, .1),(.8, .1, .1), (.8, .1, .1)]


learning_procedures = zip(names, algorithms, hyperp_sets, percentages)

data_matrices = [dist_hamming, dist_jaccard, dist_length, dist_leven]
data = zip(data_names, data_matrices)

experiments = it.product(learning_procedures, data, components)

num_experiments = len(algorithms) *  len(data_matrices) *  len(components)



performed_experiments = 0

re_generate_files = False

append = True

num_groups = 10
for g in range(num_groups):
    logger.info('Started experiment group {} of {}'.format(g+1, num_groups))
    for experiment in experiments:
        learning_procedure = experiment[0]
        data_item = experiment[1]
        comp = experiment[2]
        out_file = result_dir + 'class-' + \
                   learning_procedure[0].replace(' ', '_') + '-' + \
                   'tsne-' + \
                   data_item[0] + '-' + \
                   str(comp) + '.json'
        logger.info('Considering {}'.format(out_file))
        if os.path.isfile(out_file) and not re_generate_files and not append:
            logger.info('already exists, skipping')
        else:
            access_flag = 'w' if not append else 'a'
            if append:
                logger.info('appending to existing file')
            elif re_generate_files:
                logger.info('re-executing experiment')
            else:
                logger.info('does not exist, starting experiment')
            # perform experiment
            res = learn_experiment(data_item[1],
                                   TSNE_custom,  'tsne-' + experiment[1][0],
                                   mu,
                                   learning_procedure[1],
                                   d=comp,
                                   alpha_levels=(.2, .4, .5, .6, .8, .9),
                                   percentages=learning_procedure[3],
                                   hyperparams_set = learning_procedure[2])

            if access_flag == 'a':
                logger.info('opening result file in append')
            else:
                logger.info('creating result file')
            with open(out_file, access_flag) as f:
                f.write('\n')
                json.dump(res, f)

            logger.info('experiment finished')
        performed_experiments += 1
        perc_complete = 100 * performed_experiments / num_experiments
        logger.info('completed {:.2f}%'.format(perc_complete))
    logger.info('Ended experiment group {} of {}'.format(g+1, num_groups))

2019-06-28 15:25:04,065 - INFO - Considering ./data/classification/class-Decision_tree-tsne-hamming-2.json
2019-06-28 15:25:04,066 - INFO - already exists, skipping
2019-06-28 15:25:04,067 - INFO - completed 0.71%
2019-06-28 15:25:04,068 - INFO - Considering ./data/classification/class-Decision_tree-tsne-hamming-3.json
2019-06-28 15:25:04,069 - INFO - already exists, skipping
2019-06-28 15:25:04,070 - INFO - completed 1.43%
2019-06-28 15:25:04,071 - INFO - Considering ./data/classification/class-Decision_tree-tsne-hamming-5.json
2019-06-28 15:25:04,071 - INFO - already exists, skipping
2019-06-28 15:25:04,072 - INFO - completed 2.14%
2019-06-28 15:25:04,073 - INFO - Considering ./data/classification/class-Decision_tree-tsne-hamming-10.json
2019-06-28 15:25:04,073 - INFO - already exists, skipping
2019-06-28 15:25:04,074 - INFO - completed 2.86%
2019-06-28 15:25:04,075 - INFO - Considering ./data/classification/class-Decision_tree-tsne-hamming-30.json
2019-06-28 15:25:04,075 - INFO - al

2019-06-28 15:25:09,225 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': 50}
2019-06-28 15:25:09,229 - INFO - holdout 4 of 5
2019-06-28 15:25:09,404 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None}
2019-06-28 15:25:09,410 - INFO - holdout 5 of 5
2019-06-28 15:25:09,587 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': 50}
2019-06-28 15:25:09,591 - INFO - experiment finished
2019-06-28 15:25:09,592 - INFO - completed 7.86%
2019-06-28 15:25:09,592 - INFO - Considering ./data/classification/class-Decision_tree-tsne-length-3.json
2019-06-28 15:25:09,592 - INFO - does not exist, starting experiment
2019-06-28 15:25:09,593 - INFO - Found cached reduced data. Retrieving tsne-length
2019-06-28 15:25:09,596 - INFO - Checking alpha=0.20
2019-06-28 15:25:09,599 - INFO -

2019-06-28 15:25:15,564 - INFO - holdout 4 of 5
2019-06-28 15:25:15,708 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': 100}
2019-06-28 15:25:15,713 - INFO - holdout 5 of 5
2019-06-28 15:25:15,886 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': 50}
2019-06-28 15:25:15,889 - INFO - Checking alpha=0.40
2019-06-28 15:25:15,891 - INFO - holdout 1 of 5
2019-06-28 15:25:16,057 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None}
2019-06-28 15:25:16,062 - INFO - holdout 2 of 5
2019-06-28 15:25:16,239 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': 50}
2019-06-28 15:25:16,246 - INFO - holdout 3 of 5
2019-06-28 15:25:16,390 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'm

2019-06-28 15:25:21,856 - INFO - Checking alpha=0.50
2019-06-28 15:25:21,859 - INFO - holdout 1 of 5
2019-06-28 15:25:22,023 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None}
2019-06-28 15:25:22,029 - INFO - holdout 2 of 5
2019-06-28 15:25:22,213 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': 100}
2019-06-28 15:25:22,218 - INFO - holdout 3 of 5
2019-06-28 15:25:22,383 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None}
2019-06-28 15:25:22,391 - INFO - holdout 4 of 5
2019-06-28 15:25:22,556 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None}
2019-06-28 15:25:22,561 - INFO - holdout 5 of 5
2019-06-28 15:25:22,729 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_dep

2019-06-28 15:25:28,441 - INFO - holdout 3 of 5
2019-06-28 15:25:28,616 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None}
2019-06-28 15:25:28,622 - INFO - holdout 4 of 5
2019-06-28 15:25:28,804 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': None}
2019-06-28 15:25:28,810 - INFO - holdout 5 of 5
2019-06-28 15:25:28,979 - INFO - Learning with best hyperparams: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': 100}
2019-06-28 15:25:28,983 - INFO - Checking alpha=0.80
2019-06-28 15:25:28,987 - INFO - holdout 1 of 5
2019-06-28 15:25:29,168 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': 100}
2019-06-28 15:25:29,174 - INFO - holdout 2 of 5
2019-06-28 15:25:29,356 - INFO - Learning with best hyperparams: {'criterion': 'gini', 'm

...

/home/malchiodi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
2019-06-29 00:35:53,665 - INFO - Learning with best hyperparams: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
2019-06-29 00:35:53,718 - INFO - holdout 5 of 5
/home/malchiodi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
2019-06-29 00:35:54,508 - INFO - Learning with best hyperparams: {'C': 10, 'gamma': 30, 'kernel': 'rbf'}
/home/malchiodi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, Con

/home/malchiodi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
2019-06-29 00:36:06,622 - INFO - Learning with best hyperparams: {'C': 1, 'gamma': 30, 'kernel': 'rbf'}
2019-06-29 00:36:06,681 - INFO - holdout 2 of 5
/home/malchiodi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
2019-06-29 00:36:07,393 - INFO - Learning with best hyperparams: {'C': 1, 'gamma': 30, 'kernel': 'rbf'}
2019-06-29 00:36:07,452 - INFO - holdout 3 of 5
2019-06-29 00:36:08,148 - INFO - Learning with best hyperparams: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
2019-06-29 00:36:08,196 - INFO - holdout 4 of 5
2019-06-29 00:36:08,919 - INFO - Le

2019-06-29 00:36:22,430 - INFO - holdout 2 of 5
2019-06-29 00:36:23,152 - INFO - Learning with best hyperparams: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
2019-06-29 00:36:23,201 - INFO - holdout 3 of 5
2019-06-29 00:36:23,922 - INFO - Learning with best hyperparams: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
2019-06-29 00:36:23,970 - INFO - holdout 4 of 5
2019-06-29 00:36:24,676 - INFO - Learning with best hyperparams: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
2019-06-29 00:36:24,724 - INFO - holdout 5 of 5
/home/malchiodi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
2019-06-29 00:36:25,465 - INFO - Learning with best hyperparams: {'C': 1, 'gamma': 30, 'kernel': 'rbf'}
2019-06-29 00:36:25,519 - INFO - Checking alpha=0.50
2019-06-29 00:36:25,521 - INFO - holdout 1 of 5
2019-06-29 00:36:26

2019-06-29 00:36:40,224 - INFO - holdout 4 of 5
2019-06-29 00:36:40,970 - INFO - Learning with best hyperparams: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
2019-06-29 00:36:41,023 - INFO - holdout 5 of 5
2019-06-29 00:36:41,789 - INFO - Learning with best hyperparams: {'C': 10, 'gamma': 30, 'kernel': 'rbf'}
2019-06-29 00:36:41,857 - INFO - experiment finished
2019-06-29 00:36:41,858 - INFO - completed 95.71%
2019-06-29 00:36:41,858 - INFO - Considering ./data/classification/class-SVC_(gaussian)-tsne-length-30.json
2019-06-29 00:36:41,859 - INFO - does not exist, starting experiment
2019-06-29 00:36:41,859 - INFO - Found cached reduced data. Retrieving tsne-length
2019-06-29 00:36:41,863 - INFO - Checking alpha=0.20
2019-06-29 00:36:41,866 - INFO - holdout 1 of 5
/home/malchiodi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max

2019-06-29 00:36:55,577 - INFO - holdout 4 of 5
/home/malchiodi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/malchiodi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
2019-06-29 00:36:56,310 - INFO - Learning with best hyperparams: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
2019-06-29 00:36:56,357 - INFO - holdout 5 of 5
/home/malchiodi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
2019-06-29 00:36:57,077 - INFO - Learnin